In [1]:
import datetime
import os
import pandas as pd
import numpy as np
import sys
import math
from scipy.stats import ttest_ind_from_stats

sys.path.append('C:/workspace/repos/genie')

from utils3.data_io_utils import *

In [18]:
daihou_sql="""
select
	return_flag,
	approved_period,
	case
		when media_source in('kreditpedia',
		'cashcash_int',
		'affle_int',
		'xyads_int',
		'dokteruang_int',
		'unions',
		'intellectads_int',
		'superads_int',
		'tyroo_int',
		'pokktperformance_int',
		'Kredit All',
		'invite_friend_activity_01',
		'inmobi_int',
		'estrend') then 'BD'
		when media_source = 'Facebook Ads' then 'Facebook Ads'
		when media_source in('GooglePlay',
		'googleplay') then 'GooglePlay'
		when media_source = 'googleadwords_int' then 'googleadwords_int'
		when media_source = 'organic' then 'organic'
		else 'else'
end as channelgroup,
	groupup,
	count(distinct loan_id) as cnt_num,
	count(distinct case when late_date-effective_date = approved_period then loan_id end) dpd1
from
	(
	select
		t1.id loan_id,
		effective_date,
		return_flag,
		loan_status,
		approved_period,
		late_date,
		approved_principal,
		case
			when media_source is null
			or media_source = '' then channel
			else media_source
	end as media_source,
		case
			when "grouping" like '%ControlGroup%' then 'ControlGroup'
			when "grouping" like '%Test1%' then 'Test1'
			when "grouping" like '%Test2%' then 'Test2'
			when "grouping" like '%Test3%' then 'Test3'
			when "grouping" like '%Test4%' then 'Test4'
			else "grouping"
	end as groupup,
		case
			when loan_status = 'COLLECTION' then current_date-late_date
			else round(late_fee /(approved_principal*0.025))::int
	end as DPD
	from
		(
		select
			*
		from
			dw_gocash_go_cash_loan_gocash_core_loan
		where
			effective_date is not null
			and effective_date_accurate  >='2020-01-17 12:00:00'::timestamp
			and effective_date <= '2020-01-19'
			and loan_status not in ('DENIED',
			'RESCIND',
			'APPROVING',
			'CREATED')) t1
	left join (
		select
			*
		from
			(
			select
				order_id,
				late_date,
				row_number() over(partition by order_id
			order by
				late_date asc) as num
			from
				dw_gocash_gocash_collection_col_case
			where
				(order_status = 'COLLECTION_PAIDOFF'
				or order_status = 'COLLECTION')
				and app_id not in ('Credits',
				'KASANDAAI'))t
		where
			num = 1) t2 on
		t1.id = t2.order_id
	left join (
		select
			id,
			media_source
		from
			public.dw_gocash_go_cash_loan_gocash_core_customer ) t3 on
		t1.customer_id = t3.id)t
group by
	1,
	2,
	3,
	4
"""
daihou_data=get_df_from_pg(daihou_sql)

current Function [get_df_from_pg] run time is 5.09


### 新老首逾

In [19]:
daihou_data_new=daihou_data[daihou_data['return_flag']=='false']
daihou_data_old=daihou_data[daihou_data['return_flag']=='true']

In [20]:
daihou_data_new_sum=daihou_data_new.groupby('groupup')['dpd1','cnt_num'].sum()

In [21]:
daihou_data_new_sum=daihou_data_new_sum.reset_index()

In [30]:
daihou_data_new_sum['dpd1_rate']=daihou_data_new_sum['dpd1']/daihou_data_new_sum['cnt_num']

In [29]:
daihou_data_new_sum

,groupup,dpd1,cnt_num,dpd1_rate
0,,921,1717,0.536401
1,ControlGroup,176,443,0.397291
2,Test1,183,561,0.326203
3,Test2,74,549,0.134791
4,Test3,162,551,0.294011
5,Test4,81,641,0.126365


In [24]:
daihou_data_old_sum=daihou_data_old.groupby('groupup')['dpd1','cnt_num'].sum()
daihou_data_old_sum=daihou_data_old_sum.reset_index()
daihou_data_old_sum['dpd1_rate']=daihou_data_old_sum['dpd1']/daihou_data_old_sum['cnt_num']
daihou_data_old_sum

,groupup,dpd1,cnt_num,dpd1_rate
0,,117,226,0.517699
1,ControlGroup,412,1400,0.294286
2,Test1,444,1747,0.254150
3,Test2,49,1856,0.026401
4,Test3,0,1,0.000000
5,Test4,0,2,0.000000


In [25]:
daihou_data_new_sum2=daihou_data_new.groupby('channelgroup')['dpd1','cnt_num'].sum()

In [26]:
daihou_data_new_sum2=daihou_data_new_sum2.reset_index()

In [27]:
daihou_data_new_sum2['dpd1_rate']=daihou_data_new_sum2['dpd1']/daihou_data_new_sum2['cnt_num']
daihou_data_new_sum2

,channelgroup,dpd1,cnt_num,dpd1_rate
0,BD,175,474,0.369198
1,Facebook Ads,159,426,0.373239
2,GooglePlay,120,375,0.320000
3,else,219,382,0.573298
4,googleadwords_int,676,2105,0.321140
5,organic,248,700,0.354286


### 新用户 control vs test1

In [32]:
new_control_rate=daihou_data_new_sum.iloc[1,3]
new_control_num=daihou_data_new_sum.iloc[1,2]
new_control_var=math.sqrt(new_control_rate*(1-new_control_rate)/new_control_num)

In [33]:
new_test1_rate=daihou_data_new_sum.iloc[2,3]
new_test1_num=daihou_data_new_sum.iloc[2,2]
new_test1_var=math.sqrt(new_test1_rate*(1-new_test1_rate)/new_test1_num)

In [37]:
ttest_ind_from_stats(mean1=new_control_rate, std1=new_control_var, nobs1=new_control_num,
...                      mean2=new_test1_rate, std2=new_test1_var, nobs2=new_test1_num)

Ttest_indResult(statistic=52.295618781987805, pvalue=1.2008233752161294e-288)

### 新用户 test12 vs test34

In [41]:
new_test12_rate=(daihou_data_new_sum.iloc[2,1]+daihou_data_new_sum.iloc[3,1])/(daihou_data_new_sum.iloc[2,2]+daihou_data_new_sum.iloc[3,2])
new_test12_num=daihou_data_new_sum.iloc[2,2]+daihou_data_new_sum.iloc[3,2]
new_test12_var=math.sqrt(new_test12_rate*(1-new_test12_rate)/new_test12_num)

new_test34_rate=(daihou_data_new_sum.iloc[4,1]+daihou_data_new_sum.iloc[5,1])/(daihou_data_new_sum.iloc[4,2]+daihou_data_new_sum.iloc[5,2])
new_test34_num=daihou_data_new_sum.iloc[4,2]+daihou_data_new_sum.iloc[5,2]
new_test34_var=math.sqrt(new_test34_rate*(1-new_test34_rate)/new_test34_num)

In [42]:
ttest_ind_from_stats(mean1=new_test12_rate, std1=new_test12_var, nobs1=new_test12_num,
...                      mean2=new_test34_rate, std2=new_test34_var, nobs2=new_test34_num)

Ttest_indResult(statistic=54.571369663154165, pvalue=0.0)

### 新用户 test13 vs test24

In [43]:
new_test13_rate=(daihou_data_new_sum.iloc[2,1]+daihou_data_new_sum.iloc[4,1])/(daihou_data_new_sum.iloc[2,2]+daihou_data_new_sum.iloc[4,2])
new_test13_num=daihou_data_new_sum.iloc[2,2]+daihou_data_new_sum.iloc[4,2]
new_test13_var=math.sqrt(new_test13_rate*(1-new_test13_rate)/new_test13_num)

new_test24_rate=(daihou_data_new_sum.iloc[3,1]+daihou_data_new_sum.iloc[5,1])/(daihou_data_new_sum.iloc[3,2]+daihou_data_new_sum.iloc[5,2])
new_test24_num=daihou_data_new_sum.iloc[3,2]+daihou_data_new_sum.iloc[5,2]
new_test24_var=math.sqrt(new_test24_rate*(1-new_test24_rate)/new_test24_num)

In [44]:
ttest_ind_from_stats(mean1=new_test13_rate, std1=new_test13_var, nobs1=new_test13_num,
...                      mean2=new_test24_rate, std2=new_test24_var, nobs2=new_test24_num)

Ttest_indResult(statistic=361.94176861064255, pvalue=0.0)

### 老客户 control vs test

In [45]:
old_control_rate=daihou_data_old_sum.iloc[1,3]
old_control_num=daihou_data_old_sum.iloc[1,2]
old_control_var=math.sqrt(old_control_rate*(1-old_control_rate)/old_control_num)

old_test1_rate=daihou_data_old_sum.iloc[2,3]
old_test1_num=daihou_data_old_sum.iloc[2,2]
old_test1_var=math.sqrt(old_test1_rate*(1-old_test1_rate)/old_test1_num)

In [47]:
ttest_ind_from_stats(mean1=old_control_rate, std1=old_control_var, nobs1=old_control_num,
...                      mean2=old_test1_rate, std2=old_test1_var, nobs2=old_test1_num)

Ttest_indResult(statistic=99.59052522916106, pvalue=0.0)